<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

# Photutils

- Code: https://github.com/astropy/photutils
- Documentation: https://photutils.readthedocs.io/en/stable/
- Issue Tracker:  https://github.com/astropy/photutils/issues

<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this extended notebook, we will cover:</h2>
    
- Aperture Photometry
    - Bad pixel masking
    - Performing aperture photometry at multiple positions using multiple apertures
    - Encircled flux
    - Aperture masks

## Preliminaries

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading science data and error arrays from FITS files located in the [**data/**](data) subdirectory.  The FITS files contain 2D cutout images from the [Hubble Extreme-Deep Field (XDF)](https://archive.stsci.edu/prepds/xdf/) taken with the [Wide Field Camera 3 (WFC3)](https://www.stsci.edu/hst/instrumentation/wfc3) IR channel in the F160W filter (centered at ~1.6 $\mu m$).

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'
sci_hdulist = fits.open(sci_fn)
rms_hdulist = fits.open(rms_fn)

data = sci_hdulist[0].data.astype(float)
error = rms_hdulist[0].data.astype(float)
hdr = sci_hdulist[0].header
wcs = WCS(hdr)

In [ ]:
from astropy.visualization import simple_norm

plt.figure(figsize=(8, 8))
norm = simple_norm(data, 'sqrt', percent=99.)
plt.imshow(data, norm=norm)
plt.title('XDF F160W Cutout');

# Aperture Photometry

## Performing aperture photometry at multiple positions

In [ ]:
import astropy.units as u
from photutils.utils import calc_total_error
from photutils.aperture import CircularAperture, aperture_photometry

positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.
apertures = CircularAperture(positions, r=radius)

eff_gain = hdr['TEXPTIME']
total_error = calc_total_error(data, error, eff_gain)

unit = u.electron / u.s

phot = aperture_photometry(data << unit, apertures, error=total_error << unit)
phot

## Bad pixel masking

In [ ]:
# create a bad pixel
data2 = data.copy()
y, x = 59, 91
data2[y, x] = 100.

aperture_photometry(data2, apertures, error=total_error)

Note the large `aperture_sum` in the first source due to the bad pixel.  Now mask the bad pixel so that it does not contribute to the photometry.

In [ ]:
mask = np.zeros_like(data2, dtype=bool)
mask[y, x] = True

aperture_photometry(data2, apertures, error=total_error, mask=mask)

## Performing aperture photometry at multiple positions using multiple apertures

First define three different aperture shapes (different radii), but with the same positions.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radii = [5., 7.5, 9., 11.]
apertures = [CircularAperture(positions, r=r) for r in radii]

In [ ]:
apertures

In [ ]:
apertures[2][1]  # the aperture for the third radius and second position

In [ ]:
phot = aperture_photometry(data << unit, apertures, error=total_error << unit)
phot

The output table above now contains multiple columns for the `aperture_sum` and `aperture_sum_err` for each aperture.  The column names are appended with `_N`, where N is running index of the apertures in the input `apertures` list, i.e., the first aperture is `_0`, the second is `_1`, etc.

We can add columns to the table indicating the aperture radii.

In [ ]:
for i, rad in enumerate(radii):
    phot[f'aperture_radius_{i}'] = np.ones(len(phot)) * radii[i] * u.pix
    
phot

Or we can store them in the table metadata:

In [ ]:
for i in range(len(radii)):
    phot.meta[f'aperture_{i}'] = f'Circular aperture with r={radii[i]} pix'

In [ ]:
phot.meta

## Encircled flux

Here we want to perform aperture photometry at a single position, but with *many* apertures of different radii.

Instead of generating a big table, we'll simply loop over the apertures and extract the fluxes from individual tables.

In [ ]:
radii = np.linspace(0.1, 20, 100)  # 100 apertures
flux = []
for r in radii:
    aper = CircularAperture(positions[1], r=r)  # single position
    phot = aperture_photometry(data, aper)
    flux.append(phot['aperture_sum'][0])

In [ ]:
plt.plot(radii, flux, 'o-')
plt.title('Encircled Flux')
plt.xlabel('Radius (pixels)')
plt.ylabel('Aperture Sum ($e^{-1}/s$)');

## Aperture masks

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.
apertures = CircularAperture(positions, r=radius)

Create a list of `ApertureMask` objects using the `to_mask()` method.

In [ ]:
masks = apertures.to_mask(method='exact')

Let's plot the first one.

In [ ]:
mask = masks[0]  # the first one
plt.imshow(mask)
plt.colorbar()

The above image is a cutout of the aperture mask.

We can create an image with the aperture mask at its position.

In [ ]:
img = mask.to_image(shape=((200, 200)))
plt.imshow(img)
plt.colorbar()

We can also create a cutout from a data image over the mask domain.

In [ ]:
data_cutout = mask.cutout(data)
plt.imshow(data_cutout)
plt.colorbar()

We can also create a mask-weighted cutout from the data.  Here the circular aperture mask has been applied to the data.

In [ ]:
data_cutout_aper = mask.multiply(data)
plt.imshow(data_cutout_aper)
plt.colorbar()